<a href="https://colab.research.google.com/github/kaushaln1/ML_DM_labs/blob/main/ML/MachineLearning_HW_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%run /content/drive/MyDrive/Colab\ Notebooks/RegularizationValidation.ipynb


In [ ]:
'''
Read training and test datasets
'''
execute = True
'''
Reading the MNIST/USPS Handwritten Digits Dataset
'''

def readData(fname='ZipDigits.train'):
    '''
    Input:
        fname: name of file containing N examples, each with d attributes
    Output:
        X: N x d+1 numpy array
        y: N x 1 numpy array
    '''
    with open(fname) as f:
        X = []
        y = []
        rlines = f.readlines()
        for line in rlines:
            row = line.rstrip().split(' ')
            yval = int(float(row[0]))
            if yval in range (1,6):
              y.append(yval)
              xvals = [float(pixel) for pixel in row[1:]]
              X.append(xvals)
        X = np.array(X)
        y = np.array(y)
        y = y.reshape((y.shape[0], 1))
        print(f'X shape: {X.shape}')
        print(f'y shape: {y.shape}')
        return X, y
if execute:
    Xdigitstrain, ydigitstrain = readData('/content/drive/MyDrive/Colab Notebooks/ZipDigits.train')

    Ndigitstrain, pixels = Xdigitstrain.shape

    assert(Ndigitstrain == ydigitstrain.shape[0])

    print(f'Xdigitstrain shape {Xdigitstrain.shape}, ydigitstrain shape {ydigitstrain.shape}')


if execute:
    Xdigitstest, ydigitstest = readData('/content/drive/MyDrive/Colab Notebooks/ZipDigits.test')

    Ndigitstest, pixels = Xdigitstest.shape

    assert(Ndigitstest == ydigitstest.shape[0])

    print(f'Xdigitstest shape {Xdigitstest.shape}, ydigitstest shape {ydigitstest.shape}')

X shape: (3602, 256)
y shape: (3602, 1)
Xdigitstrain shape (3602, 256), ydigitstrain shape (3602, 1)
X shape: (988, 256)
y shape: (988, 1)
Xdigitstest shape (988, 256), ydigitstest shape (988, 1)


In [ ]:
'''
Compute the augmented matrix with features
Helper Functions
'''

def computeIntensity(X):
    '''
    Input:
        X: a 2 dimensional N x 256 numpy array
           each row contains the values of 256 pixels from a 16 x 16 grayscale image of a handwritten digit
           each pixel has an intensity value between -1 and 1
    Output:
        intensities: a 2 dimensional N x 1 numpy array
                     each row consists of a single value representing the
                     average pixel intesity of the corresponding image
                     See LFD Example 3.1
    '''
    print('computing intensity feature')
    N, d = X.shape
    print(f'Input shape {N}, {d}')
    '''
    TODO: Compute the intensity feature for N data points
    '''
    intensities = []
    for i in range(0,N):
      avg=sum(X[i])/d
      intensities.append(avg)

    intensities = np.array(intensities)
    intensities= intensities.reshape(N, 1)
    print(f'Output shape {intensities.shape}')

    return intensities


def computeSymmetry(X):
    '''
    Input:
        X: a 2 dimensional N x 256 numpy array
           each row contains the values of 256 pixels from a 16 x 16 grayscale image of a handwritten digit
           each pixel has an intensity value between -1 and 1
    Output:
        symmetries: a 2 dimensional N x 1 numpy array
                     each row consists of a single value representing the
                     "horizontal" symmetry of the 16 x 16 image about the vertical axis
                     See LFD Example 3.1
    '''
    print('computing symmetry feature')
    N, d = X.shape
    print(f'Input shape {N}, {d}')
    Ximgs = [X[n, :].reshape((16, 16)) for n in range(N)]
    Ximgs_flipped = [np.flip(Ximgs[n], axis=1) for n in range(N)]
    '''
    TODO: Compute the symmetry feature for N data points
    '''

    Ximgs = np.array(Ximgs)
    Ximgs_flipped = np.array(Ximgs_flipped)
    #print("Shpae of X img" ,np.shape(Ximgs) , np.shape(Ximgs[0]) )
    symmetries = []
    for i in range (0, N):
      avg_absolute_diff = sum(sum(abs(np.subtract(Ximgs[i] , Ximgs_flipped[i]))))/d
      #diff_array = np.subtract(Ximgs[i],Ximgs_flipped[i])
      #avg_absolute_diff =
      symmetries.append(avg_absolute_diff)

    # for i in range(N):
    #   diff = 0
    #   for j in range(16):
    #     diff+= abs(sum(np.subtract(Ximgs[i][j],Ximgs_flipped[i][j])))
    #   symmetries.append(diff)

    symmetries = np.array(symmetries)
    symmetries = symmetries.reshape(N, 1)
    print(f'Output shape {symmetries.shape}')
    return -symmetries


def computeAugmentedXWithFeatures(X):
    '''
    Input:
        X: a 2 dimensional N x 256 numpy array
           each row contains the values of 256 pixels from a 16 x 16 grayscale image of a handwritten digit
           each pixel has an intensity value between -1 and 1
    Output:
        Xaug: a 2 dimensional N x 3 numpy array
              the augmented feature matrix
              the i-th row corresponds to the i-th row of X (and image represented by it)
              the 0-th column is the column of 1s
              the 1-st column is the column of average intensities
              the 2-nd column is the column of horizontal symmetries
    '''
    N, d = X.shape
    intensity = computeIntensity(X)
    symmetry = computeSymmetry(X)
    dummy = np.ones((N, 1))
    Xaug = np.concatenate((dummy, intensity, symmetry), axis=1)
    # print(Xaug)
    print (f'Shape of augmented feature matrix: {Xaug.shape}')
    return Xaug
'''
Compute the augmented matrix with features
'''

print('Computing augmented training feature matrix')

Xaugtrain = computeAugmentedXWithFeatures(Xdigitstrain)

Naugtrain, d = Xaugtrain.shape

print('Computing augmented test feature matrix')

Xaugtest = computeAugmentedXWithFeatures(Xdigitstest)

Naugtest, d = Xaugtest.shape

'''
Create the dataset wih digits 1 and 5
'''

def indexDigits(y):
    '''
    Input:
        y: N x 1 2 dimensional numpy array; labels for handwritten digits
    Output:
        digit_idxs: a dictionary; the keys are digits 0 -- 9
                    for a digit k, digit_idxs[k] is a list identifying the rows labeled with digit k
    '''
    N = y.shape[0]
    digit_idxs = {}
    for n in range(N):
        digit = ydigitstrain[n, 0]
        if not digit in digit_idxs:
            digit_idxs[digit] = []
        digit_idxs[digit].append(n)
    return digit_idxs
'''
Construct the training and test sets for the rest of the exercises on classifying 1s vs 5s
'''

digit_idxs_train = indexDigits(Xaugtrain)
X1train = Xaugtrain[digit_idxs_train[1], :]
N1train = X1train.shape[0]
print(f'number of 1s: {N1train}')
X5train = Xaugtrain[digit_idxs_train[5], :]
N5train = X5train.shape[0]
print(f'number of 5s: {N5train}')
Xtrain = Xaugtrain[digit_idxs_train[1]+digit_idxs_train[5], :]
ytrain = np.concatenate((np.ones((N1train, 1)), -1*np.ones((N5train, 1))), axis=0)
Ntrain, d = Xtrain.shape
print(f'number of 1s and 5s: {Ntrain}')
print(f'Xtrain shape: {Xtrain.shape}, ytrain shape: {ytrain.shape}')


digit_idxs_test = indexDigits(Xaugtest)

X1test = Xaugtest[digit_idxs_test[1], :]
X5test = Xaugtest[digit_idxs_test[5], :]

Xtest = Xaugtest[digit_idxs_test[1]+digit_idxs_test[5], :]
ytest = np.concatenate((np.ones((len(digit_idxs_test[1]), 1)), -1*np.ones((len(digit_idxs_test[5]), 1))), axis=0)
Ntest, d = Xtest.shape
print(f'number of 1s and 5s: {Ntest}')
print(f'Xtest shape: {Xtest.shape}, ytest shape: {ytest.shape}')


Computing augmented training feature matrix
computing intensity feature
Input shape 3602, 256
Output shape (3602, 1)
computing symmetry feature
Input shape 3602, 256
Output shape (3602, 1)
Shape of augmented feature matrix: (3602, 3)
Computing augmented test feature matrix
computing intensity feature
Input shape 988, 256
Output shape (988, 1)
computing symmetry feature
Input shape 988, 256
Output shape (988, 1)
Shape of augmented feature matrix: (988, 3)
number of 1s: 1005
number of 5s: 556
number of 1s and 5s: 1561
Xtrain shape: (1561, 3), ytrain shape: (1561, 1)
number of 1s and 5s: 458
Xtest shape: (458, 3), ytest shape: (458, 1)


In [ ]:
from sklearn import svm
from sklearn.model_selection import GridSearchCV
import numpy as np
from sklearn import svm
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
import numpy as np

In [ ]:
svm_model = svm.SVC(kernel='linear')
param_grid = {'C': np.logspace(-3, 3, 7)}
grid_search = GridSearchCV(svm_model, param_grid, cv=5)
grid_search.fit(Xtrain, ytrain)

/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for exam

GridSearchCV(cv=5, estimator=SVC(kernel='linear'),
             param_grid={'C': array([1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02, 1.e+03])})

In [ ]:
best_C = grid_search.best_params_['C']
best_svm_model = grid_search.best_estimator_
ECV = 1 - grid_search.best_score_
print(f"Best Regularization parameter : {best_C}")
print(f"ECV for best Regularization {best_C} is : {ECV}")

Best Regularization parameter : 100.0
ECV for best Regularization 100.0 is : 0.017297452281477832


In [ ]:

best_svm_model.fit(Xtrain, ytrain)
Ein = 1 - best_svm_model.score(Xtrain, ytrain)
print(f"Ein for SVM model is : {Ein}")


Ein for SVM model is : 0.01665598975016014


/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [ ]:
Etest = 1 - best_svm_model.score(Xtest, ytest)
print(f"Etest for SVM model: {Etest}")


Etest for SVM model: 0.5873362445414847


In [ ]:

param_grid = {'svc__C':   np.logspace(-3, 3, 7)}

poly_svm_model = make_pipeline(PolynomialFeatures(degree=10), svm.SVC(kernel='linear'))

grid_search= GridSearchCV(poly_svm_model, param_grid, cv=5)
grid_search.fit(Xtrain, ytrain)

best_C_poly = grid_search.best_params_['svc__C']

best_poly_svm_model = grid_search.best_estimator_

ECV_poly = 1 - grid_search.best_score_



In [ ]:

print(f"Best regularization parameter for poly Transform data : {best_C_poly}")
print(f"ECV for best  regularization parameter: {ECV_poly}")

Best regularization parameter for poly Transform data : 1.0
ECV for best  regularization parameter: 0.017938477922503582


In [ ]:
best_poly_svm_model.fit(Xtrain, ytrain)


Ein_poly = 1 - best_poly_svm_model.score(Xtrain, ytrain)
print(f"Ein for poly Transform data: {Ein_poly}")


Ein for poly Transform data: 0.017296604740550947


/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [ ]:
Etest = 1 - best_poly_svm_model.score(Xtest, ytest)
print(f"Etest for poly Transform data: {Etest}")

Etest for poly Transform data: 0.5873362445414847
